 Install Libraries

In [ ]:
# !pip install selenium beautifulsoup4
# !pip install webdriver-manager
# !wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# !dpkg -i google-chrome-stable_current_amd64.deb
# !apt-get -f install -y
# !pip install rake-nltk -q
# !pip install spacy
# !pip install -U spacy spacy-transformers
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_trf
# !pip install keybert
# !pip install transformers torch
# !pip install langdetect
# !pip install matplotlib seaborn
# !pip install datasets
# !pip install scikit-learn tqdm
# !pip install seqeval rouge-score nltk
# !pip install spacy-lookups-data
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# !pip install transformers --upgrade
# !pip install sentence-transformers keybert yake


Import Libraries

In [4]:
import numpy as np
import pandas as pd
import json
import re
import ast
import os
import sys
import time
import random
import hashlib
import torch
import nltk
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import requests
# import imblearn


from datetime import datetime, timedelta
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from langdetect import detect
from wordcloud import WordCloud
from datasets import load_dataset, Dataset
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from flask import Flask
from flask_cors import CORS
from fastapi import FastAPI
import uvicorn
import gunicorn
from dotenv import load_dotenv
from pymongo import MongoClient
from pydantic import BaseModel
from notebook import notebookapp
from ipykernel import kernelapp as app


from transformers import (
    AutoTokenizer,
    AutoModel,
    BertForTokenClassification,
    TrainingArguments,
    Trainer
)

from sentence_transformers import SentenceTransformer, util
from rake_nltk import Rake
import yake
from keybert import KeyBERT
from spacy.lang.en.stop_words import STOP_WORDS
from tensorflow import keras
import tensorflow as tf


In [ ]:
# # Download required NLTK resources and Data
# nltk.download('punkt')
# nltk.download('punkt_tab')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

: 

Connect to Google Drive

In [ ]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# import os

# # Define the folder path in Google Drive
# folder_path = '/content/drive/My Drive/research_keyword'
# os.makedirs(folder_path, exist_ok=True)  # Ensure the folder exists

Mounted at /content/drive


Scraping content

In [5]:
# Define WebDriver options
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Define date range for scraping
start_date = datetime(2024,12,1)
end_date = datetime(2025,1,26)

# Cache directory
CACHE_DIR = "scraped_cache"
os.makedirs(CACHE_DIR, exist_ok=True)  # Ensure cache directory exists

# Cache file path
def get_cache_file_path(url):
    hash_name = hashlib.md5(url.encode()).hexdigest()  # Unique hash for each URL
    return os.path.join(CACHE_DIR, f"{hash_name}_cache.json")

# Cache expiration
CACHE_EXPIRATION_MINUTES = 2

def is_cache_valid(cache_file):
    if os.path.exists(cache_file):
        file_mtime = datetime.fromtimestamp(os.path.getmtime(cache_file))
        return (datetime.now() - file_mtime) <= timedelta(minutes=CACHE_EXPIRATION_MINUTES)
    return False

def load_cache(cache_file):
    with open(cache_file, 'r', encoding='utf-8') as f:
        return json.load(f)

def save_to_cache(cache_file, data):
    with open(cache_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)

# Scrape content function
def scrape_content(driver):
    """
    Scrapes main content from the current page, returning a list of unique text content by tag.
    """
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Remove unwanted tags
    for tag in soup(['header', 'footer', 'form', 'a', 'ul','li']):
        tag.decompose()

    # Define tags to extract
    tags_to_extract = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div']
    content_set = set()  # Use a set to keep content unique

    for tag in tags_to_extract:
        elements = soup.find_all(tag)
        for element in elements:
            text = element.get_text(separator=" ", strip=True)
            if text:  # Only add non-empty content
                content_set.add(text)

    return list(content_set)  # Convert the set back to a list

def navigate_to_next_page(driver):
    """
    Attempts to navigate to the next page.
    """
    try:
        next_button = driver.find_element(By.LINK_TEXT, 'Next')
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)
        return True
    except:
        return False

visited_links = set()

def scrape_linked_pages(soup, base_url):
    """
    Extracts and scrapes linked pages found on the current page.
    """
    page_links = [a['href'] for a in soup.find_all('a', href=True)]
    linked_page_texts = set()  # Use a set for uniqueness

    for link in page_links:
        if link.startswith("/"):
            full_url = base_url + link
        elif base_url in link:
            full_url = link
        else:
            continue

        if full_url in visited_links:
            continue
        visited_links.add(full_url)

        driver.get(full_url)
        time.sleep(1)
        linked_page_texts.update(scrape_content(driver))

    return list(linked_page_texts)  # Convert the set back to a list

def main_scrape(url):
    """
    Main function to scrape content from pages and linked pages.
    """
    driver.get(url)
    base_url = re.match(r'^https?://[^/]+', url).group(0)
    all_page_content = set()  # Use a set for uniqueness
    page_number = 1

    try:
        while True:
            print(f"Scraping page {page_number}...")

            # Scrape main page content
            page_content = scrape_content(driver)
            all_page_content.update(page_content)

            # Scrape linked pages
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            linked_texts = scrape_linked_pages(soup, base_url)

            all_page_content.update(linked_texts)

            if not navigate_to_next_page(driver):
                break

            page_number += 1
    finally:
        driver.quit()

    return list(all_page_content)  # Convert the set back to a list

# Define target URL and initiate scraping
url = "https://www.hnbfinance.lk/"
# url = "https://www.sampath.lk/"
# url = "https://www.dfcc.lk/"
# url = "https://www.combank.lk/"
# url = "https://www.boc.lk/"
# url = "https://www.hdfc.lk/"
# url = "https://www.hsbc.lk/"

# Cache file path
cache_file = get_cache_file_path(url)

if is_cache_valid(cache_file):
    print("Loading content from cache...")
    scraped_content = load_cache(cache_file)
else:
    print("Scraping content...")
    scraped_content = main_scrape(url)
    save_to_cache(cache_file, scraped_content)

# Save scraped content sentence-wise to a file
raw_file_path = os.path.join("raw_scraped_content.json")

with open(raw_file_path, 'w', encoding='utf-8') as f:
    json.dump(scraped_content, f, ensure_ascii=False, indent=2)  # Save in JSON with indentation

print(f"\n--- Scraped Content Saved to {raw_file_path} ---")

# Print the scraped content sentence-wise
print("\n--- Scraped Content (Unique Content Only) ---")
with open(raw_file_path, 'w', encoding='utf-8') as f:
    sentences = [{"sentence": sentence} for sentence in scraped_content]  # Wrap each sentence in a dictionary
    json.dump(sentences, f, ensure_ascii=False, indent=2)

# Print the saved content for reference
for sentence in scraped_content:
    print(sentence)



Scraping content...
Scraping page 1...

--- Scraped Content Saved to raw_scraped_content.json ---

--- Scraped Content (Unique Content Only) ---
1.1. Customers shall be given complete, clear concise and accurate and not misleading information about financial products/services. The information provided shall be available in languages preferred by customers, i.e., Sinhala, Tamil and English.
HNB FINANCE provides a flexible loan scheme that’s created especially for small scale enterprises.
General Savings Account Know Your Customer (KYC)
Being socially conscious is one of our core values at HNB Finance
Mr. Ruwan Manatunga Non-Independent, Non-Executive Director Mr. Ruwan Manatunga was appointed as a Non-Independent, Non -Executive Director of HNB FINANCE PLC with effect from 15th February 2021. Mr.Ruwan Manatunga | Non-Independent, Non-Executive Director Mr Ruwan Manatunga was appointed as a Non-Independent, Non-Executive Director of HNB Finance PLC with effect from 15 February 2021. Mr M

Content Cleaning

In [6]:
import json
import re

def clean_content(content_list):
    """
    Cleans the content list by removing addresses, phone numbers, and dates.
    """
    # Regex patterns for filtering
    phone_pattern = r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b'  # Matches phone numbers like 123-456-7890 or 123 456 7890
    url_pattern = r'https?://\S+|www\.\S+|\b\S+\.(?:com|org|net|edu|gov|io|lk)\b'  # Matches URLs starting with http, https, www, or ending with common domain extensions
    question_pattern = r'^.*\?$'  # Matches questions ending with a question mark
    address_pattern = r'\bNo\.?\s\d+[A-Za-z]?[,\s]?.*\b'  # Matches addresses like "No. 17, Highlevel Road, Maharagama"
    full_date_pattern = r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s?\d{1,2}[a-z]*[,\s]*\d{4}\b'  # Matches dates like "19th Dec 2024"
    year_month_pattern = r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)[a-z]*\s\d{4}\b'  # Matches "December 2021"
    time_range_pattern = r'\b\d{1,2}[:.]?\d{2}?\s?(?:am|pm)\s?(?:to|-)\s?\d{1,2}[:.]?\d{2}?\s?(?:am|pm)\b'  # Matches "9.00am to 5.00pm"
    half_sentence_pattern = r'\b\w{1,}\s[\w\s]*\b\w{1,2}\b[.]*$'  # Matches half-finished sentences
    numeric_range_pattern = r'\b\d{1,3}(?:,\d{3})*[-,]\d{1,3}(?:,\d{3})*(?:\.\d{2})?\b'
    number_pattern = r'\b\d+\b'
    #symbol_pattern = r'[\?\/\'":;\.+!%,#\$@&\*\^<>|‘’\(\)\[\]\{\}\-\“\”]'  # Matches ?, /, ', :, ;, ", ., +, !, %, #, $, @, &, *, ^, <, >, |, ‘, ’, (, ), [, ], {, }, -, “, ”
    redundant_period_pattern = r'(?<=\.)\s*\.(?:\s*\.)*'  # Matches consecutive periods like "..."
    single_letter_pattern = r'\b[a-zA-Z]\b'  # Matches single letters
    month_names_pattern = r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)\b'  # Matches month names
    weekday_pattern = r'\b(?:[Mm]onday|[Tt]uesday|[Ww]ednesday|[Tt]hursday|[Ff]riday|[Ss]aturday|[Ss]unday)\b'
    ordinal_pattern = r'\b(?:st|nd|rd|th)\b'  # Matches ordinal indicators
    title_pattern = r'\b(?:Mr|Ms|Mrs|Dr|Prof|Sir|Madam|Miss|Mx)\.?\b'  # Matches titles
    bullet_pattern = r'(?:\u2022|\u25E6|\u2023|\*|-)\s?'  # Matches common bullet points like •, *, -
    meaningless_sentence_pattern = r'\b(?:Up to|off on|valid till|discount on|limited time|offer ends)\b.*'  # Matches meaningless promotional sentences
    apostrophe_after_word_pattern = r"\b\w+’"
    am_pm_pattern = r'\b(?:AM|PM|am|pm)\b'
    all_symbols_pattern = r'[^a-zA-Z\s]'
    currency_pattern = r'\b(?:LKR|Mn|Rs|USD|EUR|GBP|INR)\b'
    sinhala_tamil_pattern = r'[\u0D80-\u0DFF\u0B80-\u0BFF]'

    stop_words = set(["a", "an", "and", "are", "as", "at", "be", "by", "for", "from", "has", "he", "in", "is", "it", "its", "of", "on", "that", "the", "to", "was", "were", "will", "with", "you"])

    cleaned_list = []
    seen_sentences = set()  # Track unique sentences

    for content in content_list:
        # Extract the sentence from the dictionary if it's a dict
        if isinstance(content, dict) and "sentence" in content:
            content = content["sentence"]

        # Remove matching patterns
        if re.search(sinhala_tamil_pattern, content):
            continue

        cleaned_content = re.sub(phone_pattern, '', content)
        cleaned_content = re.sub(url_pattern, '', cleaned_content)
        cleaned_content = re.sub(question_pattern, '', cleaned_content)
        cleaned_content = re.sub(address_pattern, '', cleaned_content)
        cleaned_content = re.sub(full_date_pattern, '', cleaned_content)
        cleaned_content = re.sub(year_month_pattern, '', cleaned_content)
        cleaned_content = re.sub(time_range_pattern, '', cleaned_content)
        cleaned_content = re.sub(half_sentence_pattern, '', cleaned_content)
        cleaned_content = re.sub(numeric_range_pattern, '', cleaned_content)
        cleaned_content = re.sub(number_pattern, '', cleaned_content)
        cleaned_content = re.sub(redundant_period_pattern, '', cleaned_content)
        cleaned_content = re.sub(single_letter_pattern, '', cleaned_content)
        cleaned_content = re.sub(month_names_pattern, '', cleaned_content)
        cleaned_content = re.sub(weekday_pattern, '', cleaned_content)
        cleaned_content = re.sub(ordinal_pattern, '', cleaned_content)
        cleaned_content = re.sub(title_pattern, '', cleaned_content)
        cleaned_content = re.sub(bullet_pattern, '', cleaned_content)
        #cleaned_content = re.sub(symbol_pattern, ' ', cleaned_content)
        cleaned_content = re.sub(meaningless_sentence_pattern, '', cleaned_content)
        cleaned_content = re.sub(apostrophe_after_word_pattern, lambda m: m.group(0)[:-1], cleaned_content)
        cleaned_content = re.sub(am_pm_pattern, '', cleaned_content)
        cleaned_content = re.sub(all_symbols_pattern, ' ', cleaned_content)
        cleaned_content = re.sub(currency_pattern, '', cleaned_content)


        # Remove extra spaces and add only non-empty content
        cleaned_content = ' '.join(cleaned_content.split())

        # Skip sentences with fewer than 5 words containing stop words
        words = cleaned_content.split()
        if len(words) < 5 and any(word.lower() in stop_words for word in words):
            continue

        # Skip sentences with only one word or two words
        if len(cleaned_content.split()) > 2 and cleaned_content not in seen_sentences:
            seen_sentences.add(cleaned_content)
            cleaned_list.append(cleaned_content)

    return cleaned_list

# Load raw content from file
raw_file_path = os.path.join("raw_scraped_content.json")
with open(raw_file_path, 'r', encoding='utf-8') as f:
    scraped_content = json.load(f)

# Clean the content
cleaned_content = clean_content(scraped_content)

# Save cleaned content to a file
cleaned_file_path = os.path.join( "cleaned_scraped_content.json")
with open(cleaned_file_path, 'w', encoding='utf-8') as f:
    json.dump(cleaned_content, f, ensure_ascii=False, indent=2)

# Save cleaned content to a CSV file
csv_file_path = os.path.join("cleaned_sentences.csv")
df = pd.DataFrame(cleaned_content, columns=["Sentence"])
df.to_csv(csv_file_path, index=False, encoding='utf-8')


# Print cleaned content in the console
print("\n--- Cleaned Content ---")
for sentence in cleaned_content:
    print(sentence)


print(f"\n--- Cleaned Content Saved to {cleaned_file_path} ---")



--- Cleaned Content ---
Customers shall be given complete clear concise and accurate and not misleading information about financial products services The information provided shall be available in languages preferred by customers Sinhala Tamil and English
HNB FINANCE provides flexible loan scheme that created especially for small scale enterprises
General Savings Account Know Your Customer KYC
Being socially conscious is one of our core values at HNB Finance
Ruwan Manatunga NonIndependent NonExecutive Director Ruwan Manatunga was appointed as NonIndependent Non Executive Director of HNB FINANCE PLC with effect from th Ruwan Manatunga NonIndependent NonExecutive Director Ruwan Manatunga was appointed as NonIndependent NonExecutive Director of HNB Finance PLC with effect from Manatunga is the Chief Risk Officer Deputy General Manager of Hatton National Bank PLC He has wealth of experience spanning over two decades in the banking and finance services sector He is Fellow Member of the Ins

Fix the Dataset

In [ ]:
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# -----------------------------
# 📌 Step 1: Load and Convert Dataset Columns Safely
# -----------------------------

# Load the dataset
file_path = "/final dataset_English.csv"
df = pd.read_csv(file_path)

# Ensure entities and extracted_keywords are in list format
def safe_literal_eval(val):
    if isinstance(val, str):  # Convert only if it's a string
        return ast.literal_eval(val)
    return val  # Return as is if already a list

df["entities"] = df["entities"].apply(safe_literal_eval)
df["extracted_keywords"] = df["extracted_keywords"].apply(safe_literal_eval)

# -----------------------------
# 📌 Step 2: Split Data into Training & Evaluation (First, Keep Everything)
# -----------------------------

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# -----------------------------
# 📌 Step 3: Balance NER and Keywords in Training Set (ONLY TRAINING DATA)
# -----------------------------

# 🔹 Extract entity types and apply oversampling/undersampling
df_train_exp = df_train.explode("entities")
df_train_exp["Entity Type"] = df_train_exp["entities"].apply(lambda x: x[1] if isinstance(x, list) else None)

# Count occurrences
entity_counts = df_train_exp["Entity Type"].value_counts()

# Oversampling for underrepresented entities
oversampler_ner = RandomOverSampler(sampling_strategy={"SERVICE": 150, "DATE": 50})
undersampler_ner = RandomUnderSampler(sampling_strategy={"ORG": min(400, entity_counts.get("ORG", 0)),
                                                          "PRODUCT": min(400, entity_counts.get("PRODUCT", 0))})

# Apply sampling
X_ner_train_over, y_ner_train_over = oversampler_ner.fit_resample(df_train_exp[["Sentence", "entities"]], df_train_exp["Entity Type"])
X_ner_train_balanced, y_ner_train_balanced = undersampler_ner.fit_resample(X_ner_train_over, y_ner_train_over)

# Restore structure & REMOVE DUPLICATE ENTITIES
def remove_duplicate_entities(entity_list):
    """
    Removes duplicate and overlapping entities within the same sentence.
    """
    unique_entities = []
    seen = set()

    for ent in entity_list:
        entity_tuple = (ent[0], ent[1], ent[2])  # (start, end, label)
        if entity_tuple not in seen:
            seen.add(entity_tuple)
            unique_entities.append(ent)

    return unique_entities

df_train_balanced = (
    X_ner_train_balanced.groupby("Sentence")
    .agg({"entities": lambda x: remove_duplicate_entities(list(x))})  # Remove duplicates
    .reset_index()
)


# 🔹 Extract keywords and apply oversampling/undersampling
df_train_exp_kw = df_train.explode("extracted_keywords")

# Count occurrences
keyword_counts = df_train_exp_kw["extracted_keywords"].value_counts()

# Oversampling for rare keywords
oversampler_kw = RandomOverSampler(sampling_strategy="minority")

# Adjust undersampling to prevent errors
adjusted_undersampling_strategy = {}
for keyword in ["DFCC", "HDFC", "HNB FINANCE"]:
    if keyword_counts.get(keyword, 0) > 20:  # Keep at least 20 samples
        adjusted_undersampling_strategy[keyword] = 20

undersampler_kw = RandomUnderSampler(sampling_strategy=adjusted_undersampling_strategy)

# Apply sampling
X_kw_train_over, y_kw_train_over = oversampler_kw.fit_resample(df_train_exp_kw[["Sentence", "extracted_keywords"]], df_train_exp_kw["extracted_keywords"])

# Apply undersampling only if required
if adjusted_undersampling_strategy:
    X_kw_train_balanced, y_kw_train_balanced = undersampler_kw.fit_resample(X_kw_train_over, y_kw_train_over)
else:
    X_kw_train_balanced, y_kw_train_balanced = X_kw_train_over, y_kw_train_over

# Restore full keyword lists per sentence (REMOVING DUPLICATES)
df_keywords_train_balanced = (
    X_kw_train_balanced.groupby("Sentence")
    .agg({"extracted_keywords": lambda x: list(set(x))})  # Remove duplicates
    .reset_index()
)

# -----------------------------
# 📌 Step 4: Save Training & Test Datasets
# -----------------------------

# Save training datasets
df_train_balanced.to_csv("/ner_train.csv", index=False)
df_keywords_train_balanced.to_csv("/keywords_train.csv", index=False)

# Save test datasets (Unchanged)
df_test.to_csv("/test_data.csv", index=False)

print("✅ Balanced datasets created! **Training set now retains more data.**")


✅ Balanced datasets created! **Training set now retains more data.**


Tokenize and convert to Hugging Face Dataset

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BertForTokenClassification, TrainingArguments, Trainer
import torch
import ast

# -----------------------------
# 📌 Step 1: Load Datasets
# -----------------------------

# Load the balanced training dataset
train_csv_path = "/keywords_train.csv"
df_train = pd.read_csv(train_csv_path)

# Load the test dataset (to be split into validation and test)
test_csv_path = "/test_data.csv"
df_test = pd.read_csv(test_csv_path)

# Ensure 'extracted_keywords' is in list format
df_train["extracted_keywords"] = df_train["extracted_keywords"].apply(ast.literal_eval)
df_test["extracted_keywords"] = df_test["extracted_keywords"].apply(ast.literal_eval)

# -----------------------------
# 📌 Step 2: Convert to Hugging Face Dataset
# -----------------------------

# Convert datasets to Hugging Face format
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

# Split test dataset into 50% validation, 50% test
dataset_split = test_dataset.train_test_split(test_size=0.5, seed=42)
val_dataset = dataset_split["train"]  # 50% of test set → Validation
test_dataset = dataset_split["test"]  # 50% of test set → Final Test Set

# -----------------------------
# 📌 Step 3: Tokenization
# -----------------------------
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

def tokenize_data(example):
    encoding = tokenizer(example["Sentence"], padding="max_length", truncation=True, max_length=128)

    # Create binary labels (1 if the token is part of an extracted keyword, else 0)
    labels = [0] * len(encoding["input_ids"])

    for keyword in example["extracted_keywords"]:
        tokens = tokenizer.tokenize(keyword)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)

        # Ensure we match full keyword tokens inside the input sequence
        for i in range(len(encoding["input_ids"]) - len(token_ids) + 1):
            if encoding["input_ids"][i:i+len(token_ids)] == token_ids:
                labels[i:i+len(token_ids)] = [1] * len(token_ids)

    encoding["labels"] = labels
    return encoding

# Apply tokenization
train_dataset = train_dataset.map(tokenize_data, remove_columns=["Sentence", "extracted_keywords"])
val_dataset = val_dataset.map(tokenize_data, remove_columns=["Sentence", "extracted_keywords"])
test_dataset = test_dataset.map(tokenize_data, remove_columns=["Sentence", "extracted_keywords"])


Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Fine Tune NER Model

In [ ]:
# import spacy

# # Ensure the correct model is loaded after installation
# try:
#     nlp = spacy.load("en_core_web_trf")
#     print("✅ Successfully loaded en_core_web_trf!")
# except Exception as e:
#     print("❌ Error loading en_core_web_trf:", e)


/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev

✅ Successfully loaded en_core_web_trf!


In [ ]:
import spacy
import pandas as pd
import ast
import random
from sklearn.model_selection import train_test_split

# -----------------------------
# 📌 Step 1: Load Datasets
# -----------------------------
train_csv_path = "/ner_train.csv"  # Update with actual path
test_csv_path = "/test_data.csv"  # Update with actual path

# Load datasets
df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

# Ensure 'entities' column is properly formatted
df_train["entities"] = df_train["entities"].apply(ast.literal_eval)
df_test["entities"] = df_test["entities"].apply(ast.literal_eval)

import random

def augment_sentence(sentence, entities):
    """
    Replace existing entities with alternative financial terms for augmentation.
    """
    replacements = {
        "HNB Finance": ["Sampath Bank", "BOC", "DFCC"],
        "Housing Loan": ["Mortgage Loan", "Personal Loan", "Auto Loan"],
        "Savings Account": ["Fixed Deposit", "Current Account"],
    }

    new_sentence = sentence
    for entity, options in replacements.items():
        if entity in sentence:
            new_sentence = new_sentence.replace(entity, random.choice(options))

    return new_sentence

# Apply augmentation to the dataset
df_train["augmented_sentence"] = df_train.apply(lambda row: augment_sentence(row["Sentence"], row["entities"]), axis=1)

# Append augmented data to training set
df_train_augmented = df_train.copy()
df_train_augmented["Sentence"] = df_train_augmented["augmented_sentence"]
df_train = pd.concat([df_train, df_train_augmented], ignore_index=True)

# -----------------------------
# 📌 Step 2: Split Test Data into Validation & Test
# -----------------------------
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42)

# -----------------------------
# 📌 Step 3: Convert to SpaCy Format (Removing Duplicates & Overlapping Entities)
# -----------------------------
def clean_entities(entities):
    """
    Removes duplicate and overlapping entities while keeping the longest entity.
    """
    entities = sorted(entities, key=lambda x: (x[0], x[1]))  # Sort by start position
    cleaned_entities = []
    prev_start, prev_end = -1, -1

    for start, end, label in entities:
        if start >= prev_end:  # Ensure no overlap
            cleaned_entities.append((start, end, label))
            prev_start, prev_end = start, end

    return cleaned_entities

def convert_to_spacy_format(df):
    """
    Convert dataset into SpaCy-compatible format:
    [('Sentence', {'entities': [(start, end, label)]})]
    """
    training_data = []
    for _, row in df.iterrows():
        sentence = row["Sentence"]
        raw_entities = row["entities"]
        cleaned_entities = clean_entities([(ent[2], ent[3], ent[1]) for ent in raw_entities])  # Remove duplicates
        training_data.append((sentence, {"entities": cleaned_entities}))
    return training_data

TRAIN_DATA = convert_to_spacy_format(df_train)
VALIDATION_DATA = convert_to_spacy_format(df_val)
TEST_DATA = convert_to_spacy_format(df_test)

# -----------------------------
# 📌 Step 4: Load & Modify Pretrained SpaCy Model
# -----------------------------
nlp = spacy.load("en_core_web_sm")  # Load SpaCy's small English model

# Ensure the NER pipeline exists
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe("ner")

# Add entity labels to the NER pipeline
for _, annotations in TRAIN_DATA:
    for ent in annotations["entities"]:
        ner.add_label(ent[2])

# -----------------------------
# 📌 Step 5: Train the NER Model
# -----------------------------
optimizer = nlp.resume_training()
random.seed(42)

for itn in range(15):  # Train for 10 epochs
    random.shuffle(TRAIN_DATA)
    losses = {}
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = spacy.training.Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.3, losses=losses)
    print(f"Epoch {itn+1} Loss: {losses}")

# -----------------------------
# 📌 Step 6: Save Fine-Tuned Model
# -----------------------------
output_path = "/finetuned_spacy_ner"
nlp.to_disk(output_path)
print(f"✅ Fine-tuned SpaCy NER model saved at: {output_path}")

# -----------------------------
# 📌 Step 7: Evaluate Model Performance on Validation & Test Data
# -----------------------------
def evaluate_ner_model(nlp_model, dataset, dataset_name):
    correct = 0
    total = 0
    for text, annotations in dataset:
        doc = nlp_model(text)
        predicted_entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        true_entities = annotations["entities"]
        total += len(true_entities)
        correct += sum(1 for ent in predicted_entities if ent in true_entities)
    accuracy = correct / total if total > 0 else 0
    print(f"📊 {dataset_name} Accuracy: {accuracy:.2%}")

# Run evaluations
evaluate_ner_model(nlp, VALIDATION_DATA, "Validation Set")
evaluate_ner_model(nlp, TEST_DATA, "Test Set")

/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Kedella Personal Loan is your ultimate partner for..." with entities "[(0, 20, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Deposits Fixed Deposits Get the value for your sav..." with entities "[(0, 25, 'PRODUCT'), (153, 162, 'GPE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Liam Anderson secured a ho

Epoch 1 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 1769.491071323599}
Epoch 2 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 1079.392685993893}
Epoch 3 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 926.8837308253997}
Epoch 4 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 779.4275509628574}
Epoch 5 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 700.156516378989}
Epoch 6 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 656.6934199016555}
Epoch 7 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 617.717840049601}
Epoch 8 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 551.0803248816462}
Epoch 9 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 543.4545600212612}
Epoch 10 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 500.21214700237425}
Epoch 11 Loss: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 466.65907945951454}
Epoch 12 Loss: {'tok2vec': 0.0, 'tagger':

Get NER Tags

In [ ]:
import spacy
import pandas as pd
import json

# -----------------------------
# 📌 Step 1: Load Fine-Tuned SpaCy NER Model
# -----------------------------
fine_tuned_model_path = "/finetuned_spacy_ner"  # Update if needed
nlp = spacy.load(fine_tuned_model_path)

# -----------------------------
# 📌 Step 2: Load Cleaned Sentences Dataset
# -----------------------------
file_path = "/cleaned_sentences.csv"  # Update if needed
df = pd.read_csv(file_path)

# -----------------------------
# 📌 Step 3: Extract Named Entities from Sentences
# -----------------------------
def extract_ner_tags(sentence):
    """Extracts NER tags using the fine-tuned SpaCy model"""
    doc = nlp(sentence)
    entities = [[ent.text, ent.label_, ent.start_char, ent.end_char] for ent in doc.ents]
    return entities

# Apply NER Extraction
df["entities"] = df["Sentence"].apply(extract_ner_tags)

# -----------------------------
# 📌 Step 4: Save Results
# -----------------------------
output_path_csv = "/ner_extracted_sentences.csv"
output_path_json = "/ner_extracted_sentences.json"

# Save as CSV
df.to_csv(output_path_csv, index=False)

# Save as JSON
with open(output_path_json, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, indent=4, ensure_ascii=False)

print(f"✅ NER tags extracted and saved: \n📂 CSV: {output_path_csv} \n📂 JSON: {output_path_json}")


/usr/local/lib/python3.11/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✅ NER tags extracted and saved: 
📂 CSV: /content/drive/My Drive/research_keyword/ner_extracted_sentences.csv 
📂 JSON: /content/drive/My Drive/research_keyword/ner_extracted_sentences.json


Fine Tune FinBERT Model

In [ ]:
from transformers import BertForTokenClassification, TrainingArguments, Trainer
import torch

# Load FinBERT model but IGNORE classifier mismatch
model = BertForTokenClassification.from_pretrained(
    "ProsusAI/finbert",
    num_labels=2,
    ignore_mismatched_sizes=True  # This will ignore the old classifier layer
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./finbert-keyword-extraction",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,  # Reduce batch size if memory issues
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset  # Using split validation dataset
)

# Train model
trainer.train()

# Save fine-tuned model
model.save_pretrained("/finbert-keyword-extraction")
tokenizer.save_pretrained("/finbert-keyword-extraction")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.047100,0.041103
2,0.028100,0.040524
3,0.013600,0.046360


('/content/drive/My Drive/research_keyword/finbert-keyword-extraction/tokenizer_config.json',
 '/content/drive/My Drive/research_keyword/finbert-keyword-extraction/special_tokens_map.json',
 '/content/drive/My Drive/research_keyword/finbert-keyword-extraction/vocab.txt',
 '/content/drive/My Drive/research_keyword/finbert-keyword-extraction/added_tokens.json',
 '/content/drive/My Drive/research_keyword/finbert-keyword-extraction/tokenizer.json')

RAKE

In [ ]:
# Load the dataset
file_path = "/ner_extracted_sentences.csv"
df = pd.read_csv(file_path)

from rake_nltk import Rake

def extract_keywords_rake(text):
    rake = Rake()
    rake.extract_keywords_from_text(text)
    return list(rake.get_ranked_phrases())

# Apply RAKE to dataset
df["rake_keywords"] = df["Sentence"].apply(extract_keywords_rake)

# Display extracted keywords
print(df[["Sentence", "rake_keywords"]].head())


                                            Sentence  \
0  In fastpaced economy housing loans have become...   
1  Miyulasi Women Savings Savings account for emp...   
2  HNB FINANCE donates consignment of road signbo...   
3  Fixed Deposits Secure your Financial Future Ou...   
4  HNB FINANCE PLC announces appointment of new B...   

                                       rake_keywords  
0  [fastpaced economy housing loans, provides fle...  
1  [conditions miyulasi women savings key facts d...  
2  [social development premier financial services...  
3  [obtain loan facilities, maintain fixed deposi...  
4  [new board member hnb finance plc hnbf recentl...  


YAKE

In [7]:
import yake

# Initialize YAKE
yake_extractor = yake.KeywordExtractor(lan="en", n=3, dedupLim=0.9, top=10, features=None)

def extract_keywords_yake(text):
    keywords = yake_extractor.extract_keywords(text)
    return [kw[0] for kw in keywords]  # Extract just the keyword strings

# Apply YAKE
df["yake_keywords"] = df["Sentence"].apply(extract_keywords_yake)

# Display YAKE results
print(df[["Sentence", "yake_keywords"]].head())


                                            Sentence  \
0  In fastpaced economy housing loans have become...   
1  Miyulasi Women Savings Savings account for emp...   
2  HNB FINANCE donates consignment of road signbo...   
3  Fixed Deposits Secure your Financial Future Ou...   
4  HNB FINANCE PLC announces appointment of new B...   

                                       yake_keywords  
0  [HNB Finance makes, Nivahana Housing Loan, fas...  
1  [HNB Finance ATM, Finance ATM card, ATM card S...  
2  [Nugegoda Police precinc, HNB FINANCE donates,...  
3  [Fixed Deposits Secure, Finance Business Act, ...  
4  [HNB FINANCE PLC, Member HNB FINANCE, FINANCE ...  


FinBERT + KeyBERT

In [ ]:
from keybert import KeyBERT
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd

# Load Fine-Tuned FinBERT Model
finbert_model_path = "/finbert-keyword-extraction"

finbert_model = AutoModel.from_pretrained(finbert_model_path)
finbert_tokenizer = AutoTokenizer.from_pretrained(finbert_model_path)

# Initialize KeyBERT with Fine-Tuned FinBERT
kw_model = KeyBERT(model=finbert_model)

def extract_keyphrases_keybert(text, top_n=5):
    """
    Extracts keyphrases using KeyBERT with the fine-tuned FinBERT model.
    """
    keyphrases = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1, 3),  # Extract unigrams, bigrams, and trigrams
        stop_words="english",
        top_n=top_n,
        use_mmr=True,  # Ensures diversity in keyphrases
        diversity=0.7
    )

    return [kp[0] for kp in keyphrases]  # Extract only keyphrases without scores

# Apply KeyBERT with Fine-Tuned FinBERT
df["keybert_keyphrases"] = df["Sentence"].apply(extract_keyphrases_keybert)

# Save results
df.to_csv("/keybert_finbert_keyphrase_extraction.csv", index=False)
print("Results saved as 'keybert_finbert_keyphrase_extraction.csv'")


Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/research_keyword/finbert-keyword-extraction and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Results saved as 'keybert_finbert_keyphrase_extraction.csv'


Embed Rank

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import ast

# Load dataset
file_path = "/keybert_finbert_keyphrase_extraction.csv"
df = pd.read_csv(file_path)

# Ensure 'yake_keywords' and 'keybert_keyphrases' are lists
df["yake_keywords"] = df["yake_keywords"].apply(ast.literal_eval)
df["keybert_keyphrases"] = df["keybert_keyphrases"].apply(ast.literal_eval)

# Load a more domain-adapted model for keyword similarity
embed_model = SentenceTransformer("msmarco-distilbert-base-v3")

def extract_embedrank_keywords(text, yake_keywords, keybert_keywords, top_n=5):
    """
    Extracts relevant keywords using sentence embeddings (EmbedRank).
    Fix: Uses only sentence-relevant keywords instead of a global dataset.
    """
    keywords = list(set(yake_keywords + keybert_keywords))  # Unique keywords per sentence

    if not keywords:  # If no keywords available
        return ["No keywords found"]

    # Compute embeddings
    text_embedding = embed_model.encode(text, convert_to_tensor=True)
    word_embeddings = embed_model.encode(keywords, convert_to_tensor=True)

    # Compute cosine similarity scores
    similarity_scores = util.pytorch_cos_sim(text_embedding, word_embeddings)[0]

    # Sort and select top-N keywords
    ranked_keywords = sorted(zip(keywords, similarity_scores.tolist()), key=lambda x: x[1], reverse=True)[:top_n]

    return [kw[0] for kw in ranked_keywords]

# Apply EmbedRank on each sentence separately
df["embedrank_keywords"] = df.apply(
    lambda row: extract_embedrank_keywords(row["Sentence"], row["yake_keywords"], row["keybert_keyphrases"], top_n=5), axis=1
)

# Save results
output_path = "/hybrid_keywords.csv"
df.to_csv(output_path, index=False)

print(f"✅ Fixed EmbedRank keywords extracted & saved successfully: {output_path}")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Fixed EmbedRank keywords extracted & saved successfully: /content/drive/My Drive/research_keyword/hybrid_keywords.csv


Scoring System

In [ ]:
import pandas as pd
import json
import nltk
import spacy
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS

# Load English NLP model for POS tagging
nlp = spacy.load("en_core_web_sm")

# Load Stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english")) | STOP_WORDS  # Combine NLTK & SpaCy stopwords

# Load the extracted keywords dataset
file_path = "/hybrid_keywords.csv"
df = pd.read_csv(file_path)

# Load the NER tagged dataset
ner_file_path = "/ner_results.csv"
df_ner = pd.read_csv(ner_file_path)

# Convert NER entities into a dictionary (Sentence → Entity List)
ner_dict = {}

for _, row in df_ner.iterrows():
    sentence = row["Sentence"].lower()  # Normalize to lowercase
    ner_entities = eval(row["entities"])  # Convert string to list
    ner_keywords = {entity[0].lower() for entity in ner_entities}  # Extract entity names in lowercase
    ner_dict[sentence] = ner_keywords


# ✅ Function to filter out stopwords and verbs
def clean_keywords(keyword_list):
    """
    Removes stopwords and verbs from keyword phrases.
    """
    filtered_keywords = set()

    for phrase in keyword_list:
        doc = nlp(phrase)  # Process phrase with SpaCy
        clean_phrase = " ".join([token.text for token in doc if token.text.lower() not in stop_words and token.pos_ != "VERB"])
        if clean_phrase:
            filtered_keywords.add(clean_phrase)  # Store only non-empty phrases

    return filtered_keywords


# ✅ Function to normalize and score keywords, with NER boosting
def score_keywords(row):
    """
    Assigns scores to keywords based on:
    - If found in all 3 extraction methods, highest score.
    - If found in 2 methods, medium score.
    - If found in 1 method, lower score.
    - If also detected in **NER**, receives **extra boost**.
    """

    # Extract and normalize keywords (convert to lowercase)
    yake_keywords = clean_keywords(set(kw.lower().strip("[]'") for kw in eval(row["yake_keywords"])))
    keybert_keywords = clean_keywords(set(kw.lower().strip("[]'") for kw in eval(row["keybert_keyphrases"])))
    embedrank_keywords = clean_keywords(set(kw.lower().strip("[]'") for kw in eval(row["embedrank_keywords"])))

    # Merge all keywords
    all_keywords = yake_keywords | keybert_keywords | embedrank_keywords

    # Get corresponding NER entities for the sentence
    sentence_ner_keywords = ner_dict.get(row["Sentence"].lower(), set())

    # Initialize keyword scores
    keyword_scores = {}

    for kw in all_keywords:
        score = 0
        if kw in yake_keywords:
            score += 3  # YAKE gets a higher weight
        if kw in keybert_keywords:
            score += 2  # KeyBERT contribution
        if kw in embedrank_keywords:
            score += 4  # EmbedRank++ gets the highest weight

        # ✅ Boost score if keyword is found in NER entities
        if kw in sentence_ner_keywords:
            score += 5  # Higher priority given to Named Entities

        keyword_scores[kw] = score

    # Sort keywords by score in descending order
    sorted_keywords = dict(sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True))

    return sorted_keywords


# Apply scoring system
df["scored_keywords"] = df.apply(score_keywords, axis=1)

# ✅ Convert to DataFrame for sorting unique keywords across all rows
keyword_scores_global = {}

for row in df["scored_keywords"]:
    for kw, score in row.items():
        if kw in keyword_scores_global:
            keyword_scores_global[kw] += score  # Aggregate scores across sentences
        else:
            keyword_scores_global[kw] = score

# Convert to DataFrame and sort globally
df_keywords_sorted = pd.DataFrame(list(keyword_scores_global.items()), columns=["Keyword", "Score"])
df_keywords_sorted = df_keywords_sorted.sort_values(by="Score", ascending=False)

# Save results as CSV and JSON
output_csv = "/final_scored_keywords.csv"
output_json = "/final_scored_keywords.json"

df_keywords_sorted.to_csv(output_csv, index=False)

with open(output_json, "w", encoding="utf-8") as f:
    json.dump(df_keywords_sorted.to_dict(orient="records"), f, indent=4)

print(f"✅ Final unique keywords ranked by score saved to: {output_csv} and {output_json}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Final unique keywords ranked by score saved to: /content/drive/My Drive/research_keyword/final_scored_keywords.csv and /content/drive/My Drive/research_keyword/final_scored_keywords.json


In [ ]:
import pandas as pd
import json

# ✅ Load the scored keywords dataset
scored_keywords_path = "/final_scored_keywords.csv"
df_keywords = pd.read_csv(scored_keywords_path)

# ✅ Load the financial vocabulary
vocab_path = "/new_financial_vocabulary.json"
with open(vocab_path, "r", encoding="utf-8") as f:
    financial_vocab = json.load(f)

# ✅ Normalize financial vocabulary (convert to lowercase for better matching)
financial_vocab_normalized = {word.lower().strip() for word in financial_vocab}

# ✅ Function to boost financial vocabulary keywords
def prioritize_financial_keywords(df):
    """
    - Boosts scores for keywords present in the financial vocabulary.
    - Keeps all other keywords with their original scores.
    - Ensures proper sorting based on boosted scores.
    """
    df["Keyword_Lower"] = df["Keyword"].str.lower().str.strip()

    # ✅ Apply a score boost only for financial vocabulary terms
    df["Boosted_Score"] = df.apply(
        lambda row: row["Score"] * 2 if row["Keyword_Lower"] in financial_vocab_normalized else row["Score"],
        axis=1
    )

    # Drop the temporary lowercase column
    df = df.drop(columns=["Keyword_Lower"])

    return df

# ✅ Apply prioritization
df_boosted = prioritize_financial_keywords(df_keywords)

# ✅ Sort by Boosted Score (Descending) to ensure financial terms get priority
df_boosted = df_boosted.sort_values(by="Boosted_Score", ascending=False)

# ✅ Save results as CSV and JSON
boosted_output_csv = "/final_prioritized_keywords.csv"
boosted_output_json = "/final_prioritized_keywords.json"

df_boosted.to_csv(boosted_output_csv, index=False)

with open(boosted_output_json, "w", encoding="utf-8") as f:
    json.dump(df_boosted.to_dict(orient="records"), f, indent=4)

print(f"✅ Prioritized keywords saved to: {boosted_output_csv} and {boosted_output_json}")


✅ Prioritized keywords saved to: /content/drive/My Drive/research_keyword/final_prioritized_keywords.csv and /content/drive/My Drive/research_keyword/final_prioritized_keywords.json


In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# import numpy as np
# import pandas as pd
# import ast

# # Load dataset
# file_path = "/hybrid_keywords.csv"
# df = pd.read_csv(file_path)

# # Convert string lists to actual lists
# df["yake_keywords"] = df["yake_keywords"].apply(ast.literal_eval)
# df["keybert_keyphrases"] = df["keybert_keyphrases"].apply(ast.literal_eval)
# df["embedrank_keywords"] = df["embedrank_keywords"].apply(ast.literal_eval)

# # ✅ Ensure keywords are unique per row
# df["all_keywords"] = df.apply(lambda row: list(set(row["yake_keywords"] + row["keybert_keyphrases"] + row["embedrank_keywords"])), axis=1)

# # ✅ Drop rows with completely empty keyword lists
# df = df[df["all_keywords"].apply(len) > 0]

# # ✅ Replace empty lists with a default placeholder
# df["all_keywords"] = df["all_keywords"].apply(lambda x: x if x else ["placeholder"])

# # Compute TF-IDF Scores
# vectorizer = TfidfVectorizer()
# tfidf_matrix = vectorizer.fit_transform([" ".join(keywords) for keywords in df["all_keywords"]])
# tfidf_scores = np.mean(tfidf_matrix.toarray(), axis=1)

# # Assign Scores
# df["final_score"] = tfidf_scores

# # Save Scored Results
# output_path = "/scored_keywords.csv"
# df.to_csv(output_path, index=False)

# print(f"✅ Scored keywords extracted & saved successfully: {output_path}")


✅ Scored keywords extracted & saved successfully: /content/drive/My Drive/research_keyword/scored_keywords.csv


In [ ]:
financial_vocabulary = [
    "interest rate","debit card", "credit card",
    "savings account", "current account", "fixed deposit",
    "credit limit", "installments", "ATM withdrawal",
    "secured loan", "unsecured loan", "auto loan", "personal loan",
    "home loan", "gold loan", "business loan", "education loan", "mortgage loan",
    "leasing", "trade finance", "microfinance", "health insurance", "life insurance", "investment banking",
    "mutual funds", "pension fund", "equity investment", "online banking",
    "mobile banking", "SMS banking", "digital wallet", "electronic fund transfer",
    "NEFT", "RTGS", "SWIFT transfer","ATM services", "Central Bank",
    "HNB FINANCE", "Commercial Bank", "DFCC Bank", "Sampath Bank", "Hatton National Bank",
    "Peoples Bank", "Bank of Ceylon", "Standard Chartered", "Citibank", "HSBC", "NDB Bank",
    "Union Bank", "Pan Asia Bank", "Cargills Bank", "NSB", "Seylan Bank", "Amana Bank",
    # Unique keywords extracted from your dataset
    "Nivahana Housing Loan", "Miyulasi Women Savings", "Auto Loan", "Finance Scheme",
    "Gold Plan", "Trust and Security", "Digital Banking", "Loan Services",
    "Customer Support", "Online Payment", "Banking Transaction", "HNB FINANCE PLC",
    "Chaminda Prabhath", "Mobile Wallet", "Merchant Banking",
    "Green Loans", "Credit Line Expansion", "Digital Ledger"
]

import json

output_path = "/new_financial_vocabulary.json"
with open(output_path, "w") as file:
    json.dump(financial_vocabulary, file, indent=4)


In [ ]:
import pandas as pd
import json

# -------------------------------
# 📌 Load Extracted Keywords Dataset
# -------------------------------
extracted_keywords_file = "/scored_keywords.csv"  # Replace with actual filename if needed
df = pd.read_csv(extracted_keywords_file)

# -------------------------------
# 📌 Load Financial Vocabulary from JSON
# -------------------------------
financial_vocab_file = "/new_financial_vocabulary.json"  # Update with actual path
with open(financial_vocab_file, "r", encoding="utf-8") as f:
    financial_vocabulary = json.load(f)

# Ensure all financial terms are in lowercase for comparison
financial_vocabulary = set([word.lower() for word in financial_vocabulary])

# -------------------------------
# 📌 Function to Assign Higher Scores to Financial Terms
# -------------------------------
def score_keywords(row):
    """
    Assigns higher scores to keywords found in the financial vocabulary.
    """
    keywords = row["all_keywords"].split(", ")  # Convert comma-separated string to a list
    scored_keywords = {}

    for keyword in keywords:
        base_score = row["final_score"]  # Original TF-IDF-based score

        # If keyword is in financial vocabulary, boost score
        if keyword.lower() in financial_vocabulary:
            scored_keywords[keyword] = base_score * 1.5  # Boost financial keywords
        else:
            scored_keywords[keyword] = base_score  # Keep same score for other terms

    return scored_keywords

# Apply scoring function to each row
df["scored_keywords"] = df.apply(score_keywords, axis=1)

# -------------------------------
# 📌 Create Final Ranked Keywords Dataset
# -------------------------------
final_keywords_df = df[["Sentence", "scored_keywords"]]

# Save as CSV
final_keywords_csv = "/final_ranked_keywords.csv"
final_keywords_df.to_csv(final_keywords_csv, index=False)

# Save as JSON
final_keywords_json = "/final_ranked_keywords.json"
final_keywords_df.to_json(final_keywords_json, orient="records", indent=4)

# ✅ DONE!
print(f"✅ Final ranked keywords saved: {final_keywords_csv}, {final_keywords_json}")


✅ Final ranked keywords saved: /content/drive/My Drive/research_keyword/final_ranked_keywords.csv, /content/drive/My Drive/research_keyword/final_ranked_keywords.json


In [ ]:
import pandas as pd
import ast
import json

# Load the CSV file
file_path = "/final_ranked_keywords.csv" # Update with your actual file path
df = pd.read_csv(file_path)

# Function to parse dictionary-like string into actual dictionary
def parse_scored_keywords(text):
    try:
        return ast.literal_eval(text)
    except (ValueError, SyntaxError):
        return {}

# Convert scored_keywords column from string to dictionary
df["scored_keywords"] = df["scored_keywords"].apply(parse_scored_keywords)

# Flatten and aggregate keyword scores
keyword_scores = {}

for keyword_dict in df["scored_keywords"]:
    for keyword, score in keyword_dict.items():
        clean_keyword = keyword.strip("[]'")  # Clean extra symbols
        if clean_keyword in keyword_scores:
            keyword_scores[clean_keyword] += score  # Aggregate scores
        else:
            keyword_scores[clean_keyword] = score

# Convert to DataFrame for sorting
df_keywords = pd.DataFrame(list(keyword_scores.items()), columns=["Keyword", "Score"])

# Sort keywords by score in descending order
df_keywords_sorted = df_keywords.sort_values(by="Score", ascending=False)

# Save sorted unique keywords as JSON and CSV
output_json ="/sorted_unique_keywords.json"
output_csv = "/sorted_unique_keywords.csv"

df_keywords_sorted.to_csv(output_csv, index=False)

with open(output_json, "w", encoding="utf-8") as f:
    json.dump(df_keywords_sorted.to_dict(orient="records"), f, indent=4)

print(f"✅ Unique keywords ranked by score saved to: {output_csv} and {output_json}")


✅ Unique keywords ranked by score saved to: /content/drive/My Drive/research_keyword/sorted_unique_keywords.csv and /content/drive/My Drive/research_keyword/sorted_unique_keywords.json


Scoring

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# import numpy as np

# # ✅ TF-IDF Scoring
# vectorizer = TfidfVectorizer()
# tfidf_matrix = vectorizer.fit_transform(df["Sentence"])
# tfidf_scores = np.array(tfidf_matrix.toarray())

# # ✅ Cosine Similarity Scoring
# sentence_model = SentenceTransformer("msmarco-distilbert-base-v3")
# text_embeddings = sentence_model.encode(df["Sentence"])
# keyword_embeddings = sentence_model.encode(df["yake_keywords"])
# cosine_scores = [np.dot(k, t) for k, t in zip(keyword_embeddings, text_embeddings)]

# # ✅ Financial Vocabulary Scoring
# with open(f"{folder_path}/financial_vocabulary.json", "r") as f:
#     financial_terms = set(json.load(f)["financial_terms"])

# def financial_score(keywords):
#     return sum(1 for kw in keywords if kw.lower() in financial_terms) / len(keywords)

# df["financial_score"] = df["yake_keywords"].apply(financial_score)

# # ✅ Final Score Calculation (Weighted Sum)
# df["final_score"] = (0.5 * tfidf_scores[:, 0]) + (0.3 * np.array(cosine_scores)) + (0.2 * df["financial_score"])

# # ✅ Select Best Keywords Based on Final Score
# df["validated_keywords"] = df.apply(lambda row: sorted(row["yake_keywords"] + row["keybert_keywords"] + row["embedrank_keywords"] + row["t5_keywords"], key=lambda x: row["final_score"], reverse=True)[:5], axis=1)

# # ✅ Save Scored Keywords
# df.to_csv(f"{folder_path}/validated_keywords.csv", index=False)
# df.to_json(f"{folder_path}/validated_keywords.json", orient="records", indent=4)
# print("✅ Keyword validation & scoring completed!")
